In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

Here we want to create a validation set from the train set. We want a 85% train and 15%, keeping the labels proportionality

In [2]:
TRAIN_CSV = 'EPIC_100_train.csv'
VAL_CSV = 'EPIC_100_validation.csv'

### Original df

In [3]:
df = pd.read_csv(TRAIN_CSV)
df.head(5)

,narration_id,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
0,P01_01_0,P01,P01_01,00:00:01.089,00:00:00.14,00:00:03.37,8,202,open door,open,3,door,3,['door'],[3]
1,P01_01_1,P01,P01_01,00:00:02.629,00:00:04.37,00:00:06.17,262,370,turn on light,turn-on,6,light,114,['light'],[114]
2,P01_01_10,P01,P01_01,00:00:23.340,00:00:24.97,00:00:26.20,1498,1572,open drawer,open,3,drawer,8,['drawer'],[8]
3,P01_01_100,P01,P01_01,00:07:57.919,00:07:59.75,00:08:00.88,28785,28852,take cup,take,0,cup,13,['cup'],[13]
4,P01_01_101,P01,P01_01,00:08:00.020,00:08:01.47,00:08:02.21,28888,28932,open cupboard,open,3,cupboard,3,['cupboard'],[3]


In [4]:
df.shape

(67217, 15)

In [5]:
df['verb_class'].apply(lambda x: int(x))

0         3
1         6
2         3
3         0
4         3
         ..
67212     8
67213     0
67214     9
67215    30
67216    30
Name: verb_class, Length: 67217, dtype: int64

In [6]:
df = df.drop(df[df['video_id'] == 'P23_04'].index)

## Create splits and new train/val dfs

In [7]:
verbs = df['verb_class'].unique()

In [8]:
df_verb_train_list = []
df_verb_val_list = []
for verb in verbs:
    if len(df[df['verb_class'] == verb]) >= 2:
        df_verb = df[df['verb_class'] == verb]
        df_verb_train, df_verb_val = train_test_split(df_verb, test_size=0.15, train_size=0.85, random_state=1, shuffle=True)
        df_verb_train_list.append(df_verb_train)
        df_verb_val_list.append(df_verb_val)
    else:
        print(f'verb {verb} skipped. Only {len(df[df["verb_class"] == verb])} values')

verb 93 skipped. Only 1 values


In [9]:
df_train = pd.concat(df_verb_train_list)
df_val = pd.concat(df_verb_val_list)

In [10]:
df_train['verb_class'].value_counts()

verb_class
0     12580
1     10381
2      5865
3      4134
4      2954
      ...  
92        3
94        2
95        1
96        1
90        1
Name: count, Length: 96, dtype: int64

We have to make the classes to be in order of [0, 1, 2, .., 96]. Now in the last labels we have [0, ..., 92, 94, 95, 96] which raises an error.

In [11]:
for num in [94, 95, 96]:
    for index in df_train['verb_class'][df_train['verb_class'] == num].index:
        df_train.loc[index, 'verb_class'] = num-1

In [12]:
for num in [94, 95, 96]:
    for index in df_val['verb_class'][df_val['verb_class'] == num].index:
        df_val.loc[index, 'verb_class'] = num-1

In [13]:
df_train['verb_class'].unique()

array([ 3,  6,  0,  5,  1,  4, 10,  7, 59, 77, 13, 23,  9, 38, 17, 28, 12,
        2, 72, 16, 35, 60,  8, 37, 18, 14, 11, 20, 39, 31, 15, 41, 22, 45,
       24, 42, 19, 34, 51, 27, 53, 52, 48, 47, 87, 64, 71, 49, 32, 21, 33,
       56, 44, 73, 25, 63, 30, 36, 82, 67, 26, 50, 89, 46, 61, 70, 76, 68,
       74, 40, 55, 86, 43, 91, 57, 66, 83, 93, 81, 29, 65, 58, 78, 62, 94,
       54, 85, 88, 69, 75, 79, 84, 80, 92, 95, 90], dtype=int64)

In [14]:
df_val['verb_class'].unique()

array([ 3,  6,  0,  5,  1,  4, 10,  7, 59, 77, 13, 23,  9, 38, 17, 28, 12,
        2, 72, 16, 35, 60,  8, 37, 18, 14, 11, 20, 39, 31, 15, 41, 22, 45,
       24, 42, 19, 34, 51, 27, 53, 52, 48, 47, 87, 64, 71, 49, 32, 21, 33,
       56, 44, 73, 25, 63, 30, 36, 82, 67, 26, 50, 89, 46, 61, 70, 76, 68,
       74, 40, 55, 86, 43, 91, 57, 66, 83, 93, 81, 29, 65, 58, 78, 62, 94,
       54, 85, 88, 69, 75, 79, 84, 80, 92, 95, 90], dtype=int64)

In [15]:
# shuffle the rows
df_train = df_train.sample(frac = 1)
df_val = df_val.sample(frac = 1)

In [16]:
print(f'df train shape: {df_train.shape}')
print(f'df val shape: {df_val.shape}')

df train shape: (56930, 15)
df val shape: (10098, 15)


In [17]:
df_train['verb_class'].value_counts()

verb_class
0     12580
1     10381
2      5865
3      4134
4      2954
      ...  
92        3
93        2
95        1
94        1
90        1
Name: count, Length: 96, dtype: int64

In [18]:
df_val['verb_class'].value_counts()

verb_class
0     2221
1     1833
2     1035
3      730
4      522
      ... 
93       1
95       1
94       1
90       1
91       1
Name: count, Length: 96, dtype: int64

In [19]:
df_val.head()

,narration_id,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
41961,P22_111_610,P22,P22_111,00:22:33.601,00:22:33.04,00:22:34.03,67652,67701,open cupboard,open,3,cupboard,3,['cupboard'],[3]
1959,P01_102_41,P01,P01_102,00:01:36.350,00:01:35.74,00:01:46.14,4787,5307,rinse colander,rinse,2,colander,39,['colander'],[39]
10350,P02_123_303,P02,P02_123,00:15:31.167,00:15:30.86,00:15:31.80,46543,46590,put plate in drainer,put-in,5,plate,2,"['plate', 'drainer']","[2, 110]"
25857,P06_110_3,P06,P06_110,00:00:10.654,00:00:07.50,00:00:10.95,375,547,put down pan,put-down,1,pan,5,['pan'],[5]
39185,P22_104_33,P22,P22_104,00:02:02.350,00:02:01.39,00:02:02.05,6069,6102,move tomato,move,11,tomato,43,['tomato'],[43]


In [20]:
df_train.to_csv('train.csv')
df_val.to_csv('val.csv')

## weight for classes exporation

In [21]:
unique_labels = np.array(df_train['verb_class'].unique())
unique_labels.sort()
all_labels = np.array(df_train['verb_class'])

In [22]:
print(unique_labels)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]


In [23]:
all_labels

array([30,  1, 44, ..., 11,  0,  4], dtype=int64)

In [24]:
class_weights = compute_class_weight(class_weight='balanced', classes=unique_labels, y=all_labels)

In [25]:
class_weights

array([4.71399709e-02, 5.71255980e-02, 1.01111822e-01, 1.43449645e-01,
       2.00751805e-01, 2.34395586e-01, 3.05680842e-01, 4.02593913e-01,
       3.76043648e-01, 4.37653752e-01, 4.47562893e-01, 4.22379511e-01,
       6.61853609e-01, 8.91760652e-01, 9.47317625e-01, 1.22524966e+00,
       1.90681940e+00, 1.71890097e+00, 1.88260582e+00, 1.87664821e+00,
       2.08077485e+00, 2.01707766e+00, 2.05197520e+00, 2.62398599e+00,
       2.62398599e+00, 3.24055100e+00, 3.01025804e+00, 3.50899901e+00,
       3.59406566e+00, 3.36943655e+00, 3.50899901e+00, 5.29482887e+00,
       4.78242608e+00, 4.49258207e+00, 4.03415533e+00, 4.90099862e+00,
       5.75748382e+00, 4.98336835e+00, 6.51671245e+00, 5.02560028e+00,
       6.05123299e+00, 7.60283120e+00, 7.70156926e+00, 9.12339744e+00,
       6.97671569e+00, 9.72165301e+00, 1.02244971e+01, 8.12357306e+00,
       7.50659283e+00, 1.09818673e+01, 8.72089461e+00, 1.37911822e+01,
       1.04038743e+01, 1.07821970e+01, 1.16278595e+01, 9.56485215e+00,
      

In [ ]:
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

In [ ]:
class_weights_tensor.dtype

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

Image reshape and save to another dict

In [4]:
from PIL import Image
import torchvision.transforms as T

In [35]:
frame = Image.open('frame_0000042685.jpg')
frame.size

(256, 256)

In [2]:
img_path = 'sample_image.jpg'

In [12]:
img = Image.open(img_path)

In [13]:
img.size

(150, 200)

In [7]:
resize = T.Resize(size=(100, 100), antialias=True)

In [8]:
img_resize = resize(img)

In [14]:
img_resize = img.resize(size=(100, 100))

In [15]:
img_resize.size

(100, 100)

In [16]:
import os

In [17]:
root = os.getcwd()
root

'C:\\Users\\34609\\VisualStudio\\TFG\\notebooks'

In [19]:
img_resize.save(r"C:\Users\34609\VisualStudio\TFG\sample.jpg")

In [20]:
dirs = os.listdir(r"C:\Users\34609\VisualStudio\TFG")
dirs

['.git',
 '.gitignore',
 '.vscode',
 'attention_zoo',
 'configs',
 'custom_sets',
 'dataset',
 'first_try.py',
 'logs',
 'main.py',
 'main_original_David.py',
 'models',
 'notebooks',
 'README.md',
 'requirements.txt',
 'resize_epic_kitchens.py',
 'tasks',
 'test.py',
 'train.py',
 'utils',
 '__pycache__']

In [33]:
joined = os.path.join("C:/Users/34609/VisualStudio/TFG/", dirs[4])

In [34]:
joined

'C:/Users/34609/VisualStudio/TFG/configs'

In [25]:
os.mkdir(r'C:\Users\34609\VisualStudio\TFG\samples')

In [27]:
if os.path.isdir(r'C:\Users\34609\VisualStudio\TFG'):
    print('Yes')

Yes
